## Open and Read the dataset

In [1]:
import csv
def read_csv(filename):
    with open(filename,'rb') as f:
        reader = csv.DictReader(f)
        return list(reader)
enrollments = read_csv('enrollments.csv')
daily_engagement = read_csv('daily_engagement.csv')
project_submissions = read_csv('project_submissions.csv')


In [3]:
# first look of two datasets

In [5]:
enrollments[0]

{'account_key': '448',
 'cancel_date': '2015-01-14',
 'days_to_cancel': '65',
 'is_canceled': 'True',
 'is_udacity': 'True',
 'join_date': '2014-11-10',
 'status': 'canceled'}

In [6]:
daily_engagement[0]

{'acct': '0',
 'lessons_completed': '0.0',
 'num_courses_visited': '1.0',
 'projects_completed': '0.0',
 'total_minutes_visited': '11.6793745',
 'utc_date': '2015-01-09'}

In [7]:
project_submissions[0]

{'account_key': '256',
 'assigned_rating': 'UNGRADED',
 'completion_date': '2015-01-16',
 'creation_date': '2015-01-14',
 'lesson_key': '3176718735',
 'processing_state': 'EVALUATED'}

## Fixing the data types

In [8]:
from datetime import datetime as dt

# Takes a date as a string, and returns a Python datetime object. 
# If there is no date given, returns None
def parse_date(date):
    if date == '':
        return None
    else:
        return dt.strptime(date, '%Y-%m-%d')
    
# Takes a string which is either an empty string or represents an integer,
# and returns an int or None.
def parse_maybe_int(i):
    if i == '':
        return None
    else:
        return int(i)

# Clean up the data types in the enrollments table
for enrollment in enrollments:
    enrollment['cancel_date'] = parse_date(enrollment['cancel_date'])
    enrollment['days_to_cancel'] = parse_maybe_int(enrollment['days_to_cancel'])
    enrollment['is_canceled'] = enrollment['is_canceled'] == 'True'
    enrollment['is_udacity'] = enrollment['is_udacity'] == 'True'
    enrollment['join_date'] = parse_date(enrollment['join_date'])
    
enrollments[0]

{'account_key': '448',
 'cancel_date': datetime.datetime(2015, 1, 14, 0, 0),
 'days_to_cancel': 65,
 'is_canceled': True,
 'is_udacity': True,
 'join_date': datetime.datetime(2014, 11, 10, 0, 0),
 'status': 'canceled'}

In [9]:
# Clean up the data types in the engagement table
for engagement_record in daily_engagement:
    engagement_record['lessons_completed'] = int(float(engagement_record['lessons_completed']))
    engagement_record['num_courses_visited'] = int(float(engagement_record['num_courses_visited']))
    engagement_record['projects_completed'] = int(float(engagement_record['projects_completed']))
    engagement_record['total_minutes_visited'] = float(engagement_record['total_minutes_visited'])
    engagement_record['utc_date'] = parse_date(engagement_record['utc_date'])
    
daily_engagement[0]

{'acct': '0',
 'lessons_completed': 0,
 'num_courses_visited': 1,
 'projects_completed': 0,
 'total_minutes_visited': 11.6793745,
 'utc_date': datetime.datetime(2015, 1, 9, 0, 0)}

In [10]:
# Clean up the data types in the submissions table
for submission in project_submissions:
    submission['completion_date'] = parse_date(submission['completion_date'])
    submission['creation_date'] = parse_date(submission['creation_date'])

project_submissions[0]

{'account_key': '256',
 'assigned_rating': 'UNGRADED',
 'completion_date': datetime.datetime(2015, 1, 16, 0, 0),
 'creation_date': datetime.datetime(2015, 1, 14, 0, 0),
 'lesson_key': '3176718735',
 'processing_state': 'EVALUATED'}

## Investigating the Data

### Data Question 1: different numbers of unique students

In [12]:
len(enrollments)

1640

In [13]:
enrollment_num_unique_students = set()
for enrollment in enrollments:
    enrollment_num_unique_students.add(enrollment['account_key'])
len(enrollment_num_unique_students)

1302

In [14]:
# The difference:some enrolled and cancelled and then re-enrolled later.

In [15]:
len(daily_engagement)

136240

In [16]:
engagement_num_unique_students = set()
for engagement in daily_engagement:
    engagement_num_unique_students.add(engagement['acct'])
len(engagement_num_unique_students)

1237

In [17]:
# one will have numbers of engagement records

In [18]:
len(project_submissions)

3642

In [19]:
submission_num_unique_students = set()
for submission in project_submissions:
    submission_num_unique_students.add(submission['account_key'])
len(submission_num_unique_students)

743

In [20]:
# same here: one could submit several projects

### Data Question 2: Unifying the Primary Key
- Column named 'account_key' in two and 'acct' in the third
- Fix: change column from 'acct' to 'account_key'

In [21]:
for engagement in daily_engagement:
    engagement['account_key'] = engagement['acct']
    del[engagement['acct']]
print daily_engagement[0]['account_key']

0


### Data Question 3: Inconsistent Unique students numbers in two tables
- More unique students in enrollment than engagement table
- find out why? 

In [22]:
for enrollment in enrollments:
    student = enrollment['account_key']
    if student not in engagement_num_unique_students:
        print enrollment
        break

{'status': 'canceled', 'is_udacity': False, 'is_canceled': True, 'join_date': datetime.datetime(2014, 11, 12, 0, 0), 'account_key': '1219', 'cancel_date': datetime.datetime(2014, 11, 12, 0, 0), 'days_to_cancel': 0}


In [23]:
# join_date = cancel_date and days_to_cancel:0
# ccount created and canceled at the same day didn't count in enrollments

In [24]:
# Above explanation need to be verified

In [25]:
num_missing_students = 0
for enrollment in enrollments:
    if (enrollment['account_key'] not in engagement_num_unique_students
        and enrollment['join_date'] != enrollment['cancel_date']):
        num_missing_students += 1
        print enrollment
    
num_missing_students

{'status': 'canceled', 'is_udacity': True, 'is_canceled': True, 'join_date': datetime.datetime(2015, 1, 10, 0, 0), 'account_key': '1304', 'cancel_date': datetime.datetime(2015, 3, 10, 0, 0), 'days_to_cancel': 59}
{'status': 'canceled', 'is_udacity': True, 'is_canceled': True, 'join_date': datetime.datetime(2015, 3, 10, 0, 0), 'account_key': '1304', 'cancel_date': datetime.datetime(2015, 6, 17, 0, 0), 'days_to_cancel': 99}
{'status': 'current', 'is_udacity': True, 'is_canceled': False, 'join_date': datetime.datetime(2015, 2, 25, 0, 0), 'account_key': '1101', 'cancel_date': None, 'days_to_cancel': None}


3

In [26]:
# Only three records found
# those are test accounts (is_udacity = True)

In [27]:
# Need to exclude those test accounts data from our further analysis

In [28]:
# Create a set of the account keys for all Udacity test accounts
udacity_test_accounts = set()
for enrollment in enrollments:
    if enrollment['is_udacity']:
        udacity_test_accounts.add(enrollment['account_key'])
len(udacity_test_accounts)

6

In [29]:
# Given some data with an account_key field
# removes any records corresponding to Udacity test accounts
def remove_udacity_accounts(data):
    non_udacity_data = []
    for data_point in data:
        if data_point['account_key'] not in udacity_test_accounts:
            non_udacity_data.append(data_point)
    return non_udacity_data

In [30]:
# Remove Udacity test accounts from all three tables
non_udacity_enrollments = remove_udacity_accounts(enrollments)
non_udacity_engagement = remove_udacity_accounts(daily_engagement)
non_udacity_submissions = remove_udacity_accounts(project_submissions)

print len(non_udacity_enrollments)
print len(non_udacity_engagement)
print len(non_udacity_submissions)

1622
135656
3634
